In [1]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU')]


In [2]:
import pandas as pd
import numpy as np
import json

In [3]:
data_path = '/home/Danny/Data-Mining/lab2/kaggle/data/'

In [4]:
json_list = list()
with open(data_path + 'tweets_DM.json' , 'r') as file:
    for line in file:
        json_list.append(json.loads(line))

In [5]:
tweet_list = list()
for json in json_list:
    tweet_id = json['_source']['tweet']['tweet_id']
    hashtags = json['_source']['tweet']['hashtags']
    hashtag = ' '.join(hashtags)
    text = json['_source']['tweet']['text']
    text = text + ' ' + hashtag
    tweet_list.append([tweet_id, text])

In [6]:
text_df = pd.DataFrame(tweet_list, columns=['tweet_id', 'text'])

In [7]:
emotion_df = pd.read_csv(data_path + 'emotion.csv')
# emotion_df

In [8]:
identification_df = pd.read_csv(data_path + 'data_identification.csv')
# identification_df

In [9]:
test_df = identification_df[identification_df['identification'] == 'test']
# test_df

In [10]:
train_df = text_df.merge(emotion_df, left_on='tweet_id', right_on='tweet_id')
# train_df

In [11]:
x_list = train_df['text'].to_list()
y_list = train_df['emotion'].to_list()

In [12]:
x_list_list = list()
for x in x_list:
    x_list_list.append(x.split())
len(x_list_list)

1455563

In [13]:
# y_list_list = list()
# for y in y_list:
#     y_list_list.append(y.split())
# len(y_list_list)

In [14]:
# x_list = list()
# for i in x:
#     x_list.append([i])

In [15]:
# y_list = list()
# for i in y:
#     y_list.append([i])

In [16]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(x_list_list, y_list, test_size=0.2, random_state=42)
train_x, valid_x, train_y, valid_y = train_test_split(train_x, train_y, test_size=0.2, random_state=42)
print(len(train_x), len(train_y))
print(len(test_x), len(test_y))
print(len(valid_x), len(valid_y))

931560 931560
291113 291113
232890 232890


In [ ]:
import kashgari
from kashgari.tasks.classification import BiLSTM_Model
kashgari.config.use_cudnn_cell = True
import logging
logging.basicConfig(level='DEBUG')

model = BiLSTM_Model()
model.fit(train_x, 
          train_y, 
          valid_x, 
          valid_y,
          epochs=10,
         )
model.save('model/BiLSTM_epoch_50')

Preparing text vocab dict: 100%|██████████| 232890/232890 [00:01<00:00, 192553.40it/s]
2020-11-29 17:12:43,840 [DEBUG] kashgari - --- Build vocab dict finished, Total: 257059 ---
2020-11-29 17:12:43,841 [DEBUG] kashgari - Top-10: ['[PAD]', '[UNK]', '[CLS]', '[SEP]', '<LH>', 'the', 'to', 'I', 'a', 'and']
Calculating sequence length: 100%|██████████| 232890/232890 [00:00<00:00, 2002573.83it/s]
2020-11-29 17:12:45,763 [DEBUG] kashgari - Calculated sequence length = 27
2020-11-29 17:12:46,874 [DEBUG] kashgari - Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, None)]            0         
_________________________________________________________________
layer_embedding (Embedding)  (None, None, 100)         25705900  
_________________________________________________________________
bidirectional (Bidirectional (None, 256)               234496    
________

Epoch 1/10
14555/14555 [==============================] - 2272s 156ms/step - loss: 1.2267 - accuracy: 0.5566 - val_loss: 1.1324 - val_accuracy: 0.5914
Epoch 2/10
14179/14555 [============================>.] - ETA: 58s - loss: 0.9393 - accuracy: 0.6643

In [ ]:
model = kashgari.utils.load_model('model/BiLSTM_epoch_50')
model.evaluate(test_x, test_y)

In [ ]:
# model = kashgari.utils.load_model('model/BERT_base_BiLSTM_Model')
# model.evaluate(test_x, test_y)

In [ ]:
# train_df

In [ ]:
# test_df

In [ ]:
test_df = test_df.merge(text_df, left_on='tweet_id', right_on='tweet_id')
# test_df

In [ ]:
text_list = test_df['text'].tolist()
# text_list

In [ ]:
predict_list = model.predict(text_list)
predict_list

In [ ]:
test_df['predict'] = predict_list
# test_df

In [ ]:
output_df = test_df[['tweet_id', 'predict']]
output_df = output_df.rename(columns={'tweet_id':'id', 'predict':'emotion'})
# output_df

In [ ]:
output_df.to_csv('../output/BiLSTM_epoch_50.csv', index=False, header=True)